# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.117167e+02     1.409521e+00
 * time: 0.7593328952789307
     1     1.114701e+01     9.445772e-01
 * time: 2.266021966934204
     2    -1.146398e+01     1.105240e+00
 * time: 2.979142904281616
     3    -3.377732e+01     7.920563e-01
 * time: 4.049831867218018
     4    -4.713506e+01     5.602961e-01
 * time: 5.14890193939209
     5    -5.683249e+01     1.904768e-01
 * time: 6.234027862548828
     6    -5.966554e+01     1.731988e-01
 * time: 6.951547861099243
     7    -6.082192e+01     8.010144e-02
 * time: 7.681707859039307
     8    -6.133789e+01     4.042113e-02
 * time: 8.39197301864624
     9    -6.164940e+01     2.835575e-02
 * time: 9.08116602897644
    10    -6.184785e+01     2.488523e-02
 * time: 9.789427995681763
    11    -6.197745e+01     2.453670e-02
 * time: 10.520870923995972
    12    -6.207084e+01     1.655848e-02
 * time: 11.254173994064331
    13    -6.214115e+01     1.169887e-02
 * time: 11.989772081375122
    14

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671069
    AtomicLocal         -18.8557655
    AtomicNonlocal      14.8522623
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485369 
    Xc                  -19.3336820

    total               -62.261666461541
